# PART 1

In [ ]:
!pip install numpy opencv-python scikit-learn matplotlib

In [ ]:
!pip install scikit-image seaborn

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
data_dir = "/Users/soardr/Desktop/VR Assign 2/data"
categories = ["with_mask", "without_mask"]
images = []
labels = []

In [ ]:
for label, category in enumerate(categories):
    folder_path = os.path.join(data_dir, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))  # Resize for uniformity
        images.append(img)
        labels.append(label)

In [ ]:
images = np.array(images)
labels = np.array(labels)

### HoG features

In [ ]:
features = []
for img in images:
    hog_feature = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9)
    features.append(hog_feature)

features = np.array(features)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.show()

### Canny Edge features

In [ ]:
def extract_edge_features(image):
    edges = cv2.Canny(image, 50, 150)
    return np.histogram(edges.ravel(), bins=256, density=True)[0]

In [ ]:
canny_feats = []
for image in images:
    canny_edge_feats = extract_edge_features(image)
    canny_feats.append(canny_edge_feats)

canny_feats = np.array(canny_feats)

## for HoG

## Models

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### SVM Classifier

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_acc:.4f}")

In [ ]:
plot_confusion_matrix(y_test, y_pred_svm, "SVM Confusion Matrix")

### MLP Classifier

In [ ]:
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)
nn_acc = accuracy_score(y_test, y_pred_nn)
print(f"Neural Network Accuracy: {nn_acc:.4f}")

In [ ]:
plot_confusion_matrix(y_test, y_pred_nn, "Neural Network Confusion Matrix")

## Comparison

In [ ]:
print("\nClassifier Comparison:")
print(f"SVM Accuracy: {svm_acc:.4f}")
print(f"Neural Network Accuracy: {nn_acc:.4f}")

## for Canny Edge

In [ ]:
X_train_canny, X_test_canny, y_train_canny, y_test_canny = train_test_split(canny_feats, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_canny = scaler.fit_transform(X_train_canny)
X_test_canny = scaler.transform(X_test_canny)

### SVM Classifier

In [ ]:
svm_model_canny = SVC(kernel='linear')
svm_model_canny.fit(X_train_canny, y_train_canny)
y_pred_canny_svm = svm_model_canny.predict(X_test_canny)
svm_acc_canny = accuracy_score(y_test_canny, y_pred_canny_svm)
print(f"SVM Accuracy Canny: {svm_acc_canny:.4f}")

In [ ]:
plot_confusion_matrix(y_test_canny, y_pred_canny_svm, "SVM Confusion Matrix for Canny")

### MLP Classifier

In [ ]:
nn_model_canny = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
nn_model_canny.fit(X_train_canny, y_train_canny)
y_pred_nn_canny = nn_model_canny.predict(X_test_canny)
nn_acc_canny = accuracy_score(y_test_canny, y_pred_nn_canny)
print(f"Neural Network Accuracy Canny: {nn_acc_canny:.4f}")

In [ ]:
plot_confusion_matrix(y_test_canny, y_pred_nn_canny, "Neural Network Confusion Matrix Canny")

# Part 2

In [ ]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

### CNN Classifier

In [ ]:
input_shape = (128, 128, 3)

#### Model

In [ ]:
from keras import layers, models, Sequential
from keras.regularizers import l2

model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

#### Load a Model

In [ ]:
from keras import models

model = models.load_model("/content/mask_detection_model_small.h5")

model.summary()

#### Training

In [ ]:
from keras.optimizers import Adam, SGD
from keras.losses import BinaryCrossentropy

optimizers = {
    "adam": Adam(learning_rate=0.001),
    "sgd": SGD(learning_rate=0.001, momentum=0.9)
}

losses = {
    "bce": BinaryCrossentropy(label_smoothing=0.10),
    "scce": "sparse_categorical_crossentropy"
}
model.compile(optimizer=optimizers["adam"], loss=losses["bce"], metrics=['accuracy'])



In [ ]:
# preprocessing:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator for real-time augmentation and memory efficiency
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values

batch_size = 32
# Load train and validation sets using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Training set
)

val_generator = train_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Validation set
)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class labels from generator
labels = train_generator.classes  # Get all training labels

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weights)

val_labels = val_generator.classes  # Get all validation labels
val_class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(val_labels), y=val_labels)
val_class_weights_dict = dict(enumerate(val_class_weights))

print("Validation Class Weights:", val_class_weights)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Keep patience at 10
    min_delta=0.001,  # Ignore very small improvements
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=20,
    class_weight=class_weights_dict,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

# Get prediction probabilities
y_pred_probs = model.predict(val_generator)
y_true = val_generator.classes

# Handle cases where model outputs two probabilities (for binary classification)
if y_pred_probs.shape[1] == 2:
    y_pred_probs = y_pred_probs[:, 1]  # Use probability of the positive class

# Compute precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_probs)

# Compute F1 scores
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)  # Avoid division by zero

# Find best threshold (maximize F1-score)
best_threshold = thresholds[np.argmax(f1_scores)]

print(f"Best Threshold: {best_threshold:.4f}")

# Apply threshold for classification
y_pred_new = (y_pred_probs > best_threshold).astype(int)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(y_pred_probs, bins=50, edgecolor="black")
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.title("Distribution of Predicted Probabilities")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions (probabilities)
y_pred = model.predict(val_generator)

# Handle cases where model outputs two probabilities (for binary classification)
if y_pred.shape[1] == 2:
    y_pred = y_pred[:, 1]  # Take probability of the positive class

# Convert probabilities to class labels using a default threshold of 0.5
y_pred_classes = (y_pred > 0.5).astype(int)

# Get true labels
y_true = val_generator.classes

# Print Classification Report
print(classification_report(y_true, y_pred_classes, target_names=['No Mask', 'Mask']))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()

In [ ]:
# save the model
model.save('mask_detection_model_small.h5')

#### Evaluating on entire dataset

In [ ]:
# load the model
from tensorflow.keras.models import load_model
# model = load_model('mask_detection_model_first.h5')

# evaluate on entire dataset
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'Face-Mask-Detection/dataset/',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")